In [7]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import rasterio

In [35]:
nir = rasterio.open('./data/2017/nir/2017.07.01.tif')
label = rasterio.open('./data/generated/bordered.tif')
X = nir.read(1)
y = label.read(1)

X = X / np.max(X)

rows, cols = X.shape
X = X.reshape((1, rows, cols, 1))

In [36]:
model = Sequential([
    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
])

In [37]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(X, y, batch_size=32)

ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 1986 target samples.